## Задание 2.

Вы поднимаетесь по лестнице. Чтобы добраться до самого верха, вам необходимо пройти n ступеней. Каждый раз, вы можете подняться либо на 1 ступеньку, либо на 2. Как много различных способов подняться на самый верх у вас есть?

Пример:
Пусть всего n = 3 ступеней, тогда:
Изначально вы можете подняться на 1 либо две ступени:
Если поднялись на одну, то дальше вы можете подняться на 2, либо два раза на одну ступень.
Если изначально поднялись на 2, то дальше вы можете подняться только на 1 ступень.
Итого, у вас 3 варианта: 1 шаг + 1 шаг + 1 шаг, 1 шаг + 2 шаша, 2 шага + 1 шаг.


## Решение

В общем случае, на каждую n-ую ступеньку можно подняться, сделав либо 1 шаг с `n-1` ступеньки, либо 2 шага с `n-2` ступеньки.

Подняться на n-1 ступеньку можно аналогичным образом: либо сделать 1 шаг с `n-2` ступеньки, либо сделать 2 шага с `n-3` ступеньки.

Получается, что в общем случае, если `F(n)` - количество способов подняться на n ступеньку, то `F(n)` задаётся соотношением:

`F(n) = F(n-2) + F(n-1)`. 

Номер ступеньки строго положительный `(n = 1...N)`, поэтому есть два краевых случая:
- `F(0) = 1` (чтобы остаться на земле есть только один способ - не подниматься)
- `F(1) = 1` (подняться на первую ступеньку можно только шагнув на неё 1 шаг с демли) 

Решение задачи сводится к нахождению n-го числа Фибоначчи

### Комментарий

Помимо `итеративного алгоритма` есть аналитический (называется `Формула Бине`), но он работает корректно только до определённых размеров N (N<70).
Однако с его помощью можно чуть оптимизировать итоговое решение, потому что сложность по времени формулы Бине - `O(1)`

Моя итоговая функция будет содержать в себе два метода: для малых N будет считаться по `формуле Бине`, для бОльших N - через `итеративный алгоритма`.
Краевые случаи F(0) = 1 и F(1) = 1 в коде не прописаны, т.к. Формула Бине их учитывает.
Да, при разовом запуске Бине не сильно по времени выиграет, потому что итеративный и так будет быстро считаться, но если нам нужно сделать миллиард запусков, то выигрыш во времени будет приличный

Тестировал на N=68, 1 миллиадр итераций:
- Итеративный: 2735.65 секунд
- Бине: 1108.57 секунд

Разница на лицо

In [1]:
import time
from functools import wraps
import math

# если нужно запринтовать длинную строку. Иначе будет ошибка ValueError: Exceeds the limit (4300) for integer string conversion;
# import sys
# sys.set_int_max_str_digits(10**6)  

In [2]:
def timing(func):
    '''Вычисляет время работы функции''' 
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()  
        result = func(*args, **kwargs)  
        end_time = time.time()
        elapsed_time = end_time - start_time  
        print(f"Function {func.__name__} took {elapsed_time:.9f} seconds")
        return result
    return wrapper

In [3]:
def get_fibonacci_binet(N: int) -> int:
    N += 1 # т.к. числа фиб. начинаются с 1, а ступеньки у нас с 0 (1-е число это 0-я ступенька)
    phi = (1 + math.sqrt(5)) / 2 
    psi = (1 - math.sqrt(5)) / 2
    return round((phi ** N - psi ** N) / math.sqrt(5))

def get_fibonacci_iter(N: int) -> int:
    n_minus_2, n_minus_1 = 1, 1 # F(0), F(1)
    for _ in range(2, N + 1):
        n_minus_1, n_minus_2 = n_minus_2 + n_minus_1, n_minus_1 # F(n) = F(n-1) + F(n-2)
    return n_minus_1

@timing
def get_number_options_to_climb(N: int) -> int:
    '''N - количество ступеней'''
    if not isinstance(N, int) or N < 0:
        raise ValueError('N should be a natural number or 0.')
    if N < 69:
        return get_fibonacci_binet(N)
    else:
        return get_fibonacci_iter(N)

In [4]:
stairs_num = 10
print(get_number_options_to_climb(stairs_num))

Function get_number_options_to_climb took 0.000992060 seconds
89


## Сложность

Сложность по времени в худшем случае (N >= 69) - `O(N)`, в лучшем случае (N < 69) - `O(1)`.

Сложность по памяти `O(1)`

## Тесты

1. Нулевая ступенька. Краевой случай, который должен быть обработан правильно
2. Первая ступенька. Также краевой случай
3. 68 и 69 ступеньки - проверка, на то что оба алгоритма в моей реализации корректно считают корректно считает 
4. Проверка на больших данных
5. Проверка на типы данных

Для сравнения использую функцию `fibonacci` из библиотеки `sympy`. Входное значение для `fibonacci()` нужно указывать на 1 больше, так как числа Фиб. начинаются с 1, а ступеньки у нас с 0 (1-е число это 0-я ступенька)
Например, 3-е число Фибоначчи равно 2, а для третьей ступеньки ответ 3 (это 4-е число Фибоначчи)

In [5]:
import pytest
from sympy import fibonacci # для референсного значения

@timing
def test_get_number_options_to_climb():
    assert get_number_options_to_climb(0) == 1, "Failed on 0 steps"
    assert get_number_options_to_climb(1) == 1, "Failed on 1 step"
    assert get_number_options_to_climb(68) == fibonacci(69), "Failed on 68 step"
    assert get_number_options_to_climb(69) == fibonacci(70), "Failed on 69 step"
    assert get_number_options_to_climb(100500) == fibonacci(100501), "Failed on 100500 step"


    # неправильный тип данных
    with pytest.raises(ValueError) as excinfo:
        get_number_options_to_climb(-1)
    assert str(excinfo.value) == "N should be a natural number or 0.", "Failed on negative input"
    
    with pytest.raises(ValueError) as excinfo:
        get_number_options_to_climb(3.5) 
    assert str(excinfo.value) == "N should be a natural number or 0.", "Failed on float input"
    
    with pytest.raises(ValueError) as excinfo:
        get_number_options_to_climb("a")
    assert str(excinfo.value) == "N should be a natural number or 0.", "Failed on string input"

    with pytest.raises(ValueError) as excinfo:
        get_number_options_to_climb(None)
    assert str(excinfo.value) == "N should be a natural number or 0.", "Failed on None input"

    with pytest.raises(ValueError) as excinfo:
        get_number_options_to_climb([14])
    assert str(excinfo.value) == "N should be a natural number or 0.", "Failed on list input"

    print("All tests passed.")

In [6]:
test_get_number_options_to_climb()

Function get_number_options_to_climb took 0.000000000 seconds
Function get_number_options_to_climb took 0.000000000 seconds
Function get_number_options_to_climb took 0.000000000 seconds
Function get_number_options_to_climb took 0.000000000 seconds
Function get_number_options_to_climb took 0.094082355 seconds
All tests passed.
Function test_get_number_options_to_climb took 0.099005699 seconds
